In [192]:
import re
from gurobipy import *
import numpy as np
import itertools
import time
import linecache as lc

In [181]:
def read_file(filename):
    with open(filename, 'r') as file:
        content = file.read()  # Open the file in read mode

    data = {}  # Initialize data as an empty dictionary

    # Each file has number of inputs under which data is stored as a matrix
    numerical_data = re.findall(r'\[([\d\s.]+)\]', content)  # Extract values following each header as numerical values and remove '\t's and '\n's

    sections = ['Orders', 'Allocations', 'DistanceShelfShelf', 'DistancePackagingShelf', 'FullDistanceMatrixMetres']

    for i, section in enumerate(sections):
        data[section] = []
        lines = numerical_data[i].strip().split('\n')
        for line in lines:
            values = line.strip().split()  # Split using spaces
            data[section].append([int(val) for val in values])  # Assuming all values are integers

    return data

def allocation_from_txt(file):
    allocation_vector = lc.getline(file, 1).split(" ")[:-1]
    allocation_vector = [int(prod) for prod in allocation_vector]
    return np.array(allocation_vector)

# Reading in data and running the entire function

In [182]:
filename = r"Data_Xpress_FullDist_Metres.txt"
data = read_file(filename)

# Extracting data into arrays
Orders = data.get('Orders')
Allocations = np.asarray(data.get('Allocations')[0])
DistanceShelfShelf = np.asarray(data.get('DistanceShelfShelf'))
FullDistanceMatrix = np.asarray(data.get('FullDistanceMatrixMetres'))

NbShelves = 96
Shelves = range(1, NbShelves + 1)

FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 1)
FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 0)


# testing for allocation vector where every shelf is filled
allocations_full = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56, 
                    83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
                    90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
                    21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
                    54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
                    1, 33, 59, 42, 52,  45, 79, 39, 68, 73, 53]


# testing where only some of the extra shelves are filled
allocations_mid_full = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56, 
                        83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
                        90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
                        21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
                        54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
                        1, 33, 59, 42, 52,  45, 79,  0, 68,  0, 53]


In [183]:
Allocations

array([45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56,
       83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
       90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
       21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
       54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
        1, 33, 59, 42, 52,  0,  0,  0,  0,  0,  0])

In [184]:
def find_closest_product(current_shelf, order, distances):
    closest_shelf = None
    min_distance = float(1e6)
    for shelf in order:
        if distances[current_shelf][shelf] <= min_distance:
            closest_shelf = shelf
            min_distance = distances[current_shelf][shelf]
    return closest_shelf

def generate_order_lists(order):
    ''' 
    Takes a list with tuple elements and returns a list of lists 
    with all possible combinations of individual elements individual elements.
    '''
    order_lists = []
    tuple_indices = [i for i, item in enumerate(order) if isinstance(item, tuple)]
    for combination in itertools.product(*[order[i] for i in tuple_indices]):
        new_order = order.copy()
        for i, index in enumerate(tuple_indices):
            new_order[index] = combination[i]
        order_lists.append(new_order)
    return order_lists

def greedy_order_route(order, distances):
    ''' 
    Uses a greedy method of calculating the minimum distance. 

    Function has been split into two if statements to consider
    cases of orders where products are contained on more than one shelf.

    If products are contained on more than one shelf, the function constructs 
    a route with all possible shelf combinations and chooses the one with 
    the shortest distance.

    Returns a list containing the route and the total distance for the order.
    '''

    # If all products in the order are contained on one shelf only
    if not any(isinstance(product, tuple) for product in order):
        visited = [0]
        current_position = 0  
        for k in range(len(order)):
                closest_product = find_closest_product(current_position, order, distances)
                visited.append(closest_product)
                order.remove(closest_product)
                current_position = closest_product
        visited.append(0)
        OrderDistance = 0
        for i in range(len(visited) - 1):
            OrderDistance += distances[visited[i]][visited[i+1]]
        order_distance_final = OrderDistance
        visited_final = visited

    # If one or more products in the order are contained on more than one shelf
    elif any(isinstance(product, tuple) for product in order):
        order_combinations = generate_order_lists(order) # create new orders with all possible combinations from tuples
        order_routes = []   # initialise a list of routes for all combinations                                 
        order_distances = [] # initialise a list of distances for all combinations
        
        # loop over all combinations
        for order in order_combinations:
            visited = [0]
            current_position = 0  
            for k in range(len(order)):
                closest_product = find_closest_product(current_position, order, distances)
                visited.append(closest_product)
                order.remove(closest_product)
                current_position = closest_product
            visited.append(0)
            order_routes.append(visited) # add the route for the combination to the list of routes
            OrderDistance = 0
            for i in range(len(visited) - 1):
                OrderDistance += distances[visited[i]][visited[i+1]]
            order_distances.append(OrderDistance) # add the distance for the combination to the list of distances
        
        # select the order with the shortest distance among the combinations
        min_idx = order_distances.index(min(order_distances))
        visited_final = order_routes[min_idx] 
        order_distance_final = order_distances[min_idx]
            
    return visited_final, order_distance_final # return order route and distance

def convert_orders_to_shelf_indices(allocations):
    ''' 
    This function takes the allocation vector and returns an 
    order matrix with shelf indices instead of product indices.
    '''
    product_to_shelf = {}
    for shelf_index, product in enumerate(allocations):
        if product != 0:  # Check if the element is not zero
            if product not in product_to_shelf:
                product_to_shelf[product] = [shelf_index + 1]  # Initialize with a list containing the current shelf index
            else:
                # If the product already exists in the dictionary, append the new shelf index to the list
                product_to_shelf[product].append(shelf_index + 1)

    # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
    product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

    OrdersByShelf = []
    for order in Orders:
        order_shelf_indices = []
        for product_index in order:
            if product_index in product_to_shelf_tuples:
                shelf_indices = product_to_shelf_tuples[product_index]
                order_shelf_indices.append(shelf_indices)
            else:
                order_shelf_indices.append(0)  # Product not found in allocation matrix
        OrdersByShelf.append(order_shelf_indices)

    return OrdersByShelf



def q1_function(allocation_vector, distance_matrix):

    OrdersByShelf = convert_orders_to_shelf_indices(allocation_vector)
    TotalDistance = 0           # initialise counter for total distance
    DistancesPerOrder = []      # initalise list to contain the distances for each order 
    routes = []                 # intialise list to contain the routes for each order
    for order in OrdersByShelf:
        visited_order_route, visited_order_dist = greedy_order_route(order, distance_matrix)
        routes.append(visited_order_route)
        DistancesPerOrder.append(visited_order_dist)
        TotalDistance += visited_order_dist

    # Replace DistancesPerOrder in the return statement with this if you want sorted distances to be returned
    SortedDistancesPerOrder = sorted(DistancesPerOrder, reverse=True)

    # The indices corresponding to the longest orders (descending order) 
    idx_longest_orders = sorted(range(len(DistancesPerOrder)), key=lambda i: DistancesPerOrder[i], reverse=True)

    # List of routes ordered by distance (descending)
    OrderedRoutes = [routes[i] for i in idx_longest_orders]
    
    return TotalDistance, DistancesPerOrder, SortedDistancesPerOrder, idx_longest_orders, OrderedRoutes

In [185]:
### TEST CASE ON CURRENT ALLOCATION ###
TotalDistance, DistancesPerOrder, SortedDistancesPerOrder, idx_longest_orders, routes = q1_function(Allocations)

#### Obtaining total distances for different allocations

In [186]:
print(f"Distance for the current allocation: {q1_function(Allocations)[0]} metres.")
print(f"Distance for all shelves filled: {q1_function(allocations_full)[0]} metres.")
print(f"Distance for some repeated products and some empty shelves: {q1_function(allocations_mid_full)[0]} metres.")

Distance for the current allocation: 290706 metres.
Distance for all shelves filled: 289200 metres.
Distance for some repeated products and some empty shelves: 289728 metres.


## The functions contained within the distance function (for testing and sanity checks)

In [187]:
# def find_closest_product(current_position, products, distances):
#     closest_product = None
#     min_distance = float('inf')
#     for product in products:
#         if distances[current_position][product] <= min_distance:
#             closest_product = product
#             min_distance = distances[current_position][product]
#     return closest_product

# def generate_order_lists(order):
#     ''' 
#     Takes a list with tuple elements and returns a list of lists 
#     with all possible combinations of individual elements individual elements.
#     '''
#     order_lists = []
#     tuple_indices = [i for i, item in enumerate(order) if isinstance(item, tuple)]
#     for combination in itertools.product(*[order[i] for i in tuple_indices]):
#         new_order = order.copy()
#         for i, index in enumerate(tuple_indices):
#             new_order[index] = combination[i]
#         order_lists.append(new_order)
#     return order_lists

# def greedy_order_route(order, distances):
#     ''' 
#     Uses a greedy method of calculating the minimum distance. 

#     Function has been split into two if statements to consider
#     cases of orders where products are contained on more than one shelf.

#     If products are contained on more than one shelf, the function constructs 
#     a route with all possible shelf combinations and chooses the one with 
#     the shortest distance.

#     Returns a list containing the route and the total distance for the order.
#     '''

#     # If all products in the order are contained on one shelf only
#     if not any(isinstance(product, tuple) for product in order):
#         visited = [0]
#         current_position = 0  
#         for k in range(len(order)):
#                 closest_product = find_closest_product(current_position, order, distances)
#                 visited.append(closest_product)
#                 order.remove(closest_product)
#                 current_position = closest_product
#         visited.append(0)
#         OrderDistance = 0
#         for i in range(len(visited) - 1):
#             OrderDistance += distances[visited[i]][visited[i+1]]
#         order_distance_final = OrderDistance
#         visited_final = visited

#     # If one or more products in the order are contained on more than one shelf
#     elif any(isinstance(product, tuple) for product in order):
#         order_combinations = generate_order_lists(order) # create new orders with all possible combinations from tuples
#         order_routes = []   # initialise a list of routes for all combinations                                 
#         order_distances = [] # initialise a list of distances for all combinations
        
#         # loop over all combinations
#         for order in order_combinations:
#             visited = [0]
#             current_position = 0  
#             for k in range(len(order)):
#                 closest_product = find_closest_product(current_position, order, distances)
#                 visited.append(closest_product)
#                 order.remove(closest_product)
#                 current_position = closest_product
#             visited.append(0)
#             order_routes.append(visited) # add the route for the combination to the list of routes
#             OrderDistance = 0
#             for i in range(len(visited) - 1):
#                 OrderDistance += distances[visited[i]][visited[i+1]]
#             order_distances.append(OrderDistance) # add the distance for the combination to the list of distances
        
#         # select the order with the shortest distance among the combinations
#         min_idx = order_distances.index(min(order_distances))
#         visited_final = order_routes[min_idx] 
#         order_distance_final = order_distances[min_idx]
            
#     return visited_final, order_distance_final # return order route and distance

# def convert_orders_to_shelf_indices(allocations):
#     ''' 
#     This function takes the allocation vector and returns an 
#     order matrix with shelf indices instead of product indices.
#     '''
#     product_to_shelf = {}
#     for shelf_index, product in enumerate(allocations):
#         if product != 0:  # Check if the element is not zero
#             if product not in product_to_shelf:
#                 product_to_shelf[product] = [shelf_index + 1]  # Initialize with a list containing the current shelf index
#             else:
#                 # If the product already exists in the dictionary, append the new shelf index to the list
#                 product_to_shelf[product].append(shelf_index + 1)

#     # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
#     product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

#     OrdersByShelf = []
#     for order in Orders:
#         order_shelf_indices = []
#         for product_index in order:
#             if product_index in product_to_shelf_tuples:
#                 shelf_indices = product_to_shelf_tuples[product_index]
#                 order_shelf_indices.append(shelf_indices)
#             else:
#                 order_shelf_indices.append(0)  # Product not found in allocation matrix
#         OrdersByShelf.append(order_shelf_indices)

#     return OrdersByShelf



# allocation_vector = Allocations[0] # change this to test different allocations

# OrdersByShelf = convert_orders_to_shelf_indices(allocation_vector) 
# TotalDistance = 0           # initialise counter for total distance
# DistancesPerOrder = []      # initalise list to contain the distances for each order 
# routes = []                 # intialise list to contain the routes for each order
# for order in OrdersByShelf:
#     visited_order_route, visited_order_dist = greedy_order_route(order, FullDistanceMatrix)
#     routes.append(visited_order_route)
#     DistancesPerOrder.append(3*visited_order_dist)
#     TotalDistance += 3*visited_order_dist

# # Replace DistancesPerOrder in the return statement with this if you want sorted distances to be returned
# SortedDistancesPerOrder = sorted(DistancesPerOrder, reverse=True)

# # The indices corresponding to the longest orders (descending order) 
# idx_longest_orders = sorted(range(len(DistancesPerOrder)), key=lambda i: DistancesPerOrder[i], reverse=True)

# Local Search

In [188]:
def find_new_shelf_prod(shelf, allocations):
    '''
    Moves item across the corridor
    '''
    if (shelf <= 24) or (49 <= shelf <= 72): 
        new_shelf = shelf + 24
    elif (25 <= shelf <= 48) or (73 <= shelf <= 96):
        new_shelf = shelf - 24
    new_prod = allocations[new_shelf-1]

    return new_prod, new_shelf

In [189]:
def swap_prods_worst_route_neighbourhoods(initial_allocation, max_ind):
    start_time = time.time()
    init_alloc = initial_allocation.copy()
    orders_shelf = convert_orders_to_shelf_indices(init_alloc)
    init_result = q1_function(init_alloc)
    init_distance, sorted_dist, idx_sorted_dist, sorted_routes = init_result[0], init_result[2], init_result[3], init_result[4]
    neighbourhoods = []
    neigh_dists = []
    print(f"Initial distance : {init_distance}")

    #loops over the orders with the highest distances walked up to kth top orders
    for k in idx_sorted_dist[:max_ind+1]:
        #finds the products and shelves corresponding with that order
        prods = Orders[k]
        shelfs = orders_shelf[k]

        # removes 0s from the shelfs and product arrays
        shelfs_nonzero = [i for i in shelfs if i != 0]
        prods_nonzero = [i for i in prods if i != 0]

        #loops over shelves associated with the long distance order
        for i in range(len(shelfs_nonzero)):

            #takes a corresponding product and shelf from the arrays we are looking at
            prods_temp, shelfs_temp = prods_nonzero.copy(), shelfs_nonzero.copy()
            p_fix, s_fix = prods_nonzero[i], shelfs_nonzero[i]

            #removes them from the arrays
            prods_temp.remove(p_fix), shelfs_temp.remove(s_fix)

            #iterates over the length of the new array
            for j in range(len(prods_temp)):

                #resets the initial allocation
                update_alloc = init_alloc.copy()

                #looks to change one of the products in the original order (minus the one we removed)
                p_change, s_change = prods_temp[j], shelfs_temp[j]

                #switches product across the corridor
                p_new, s_new = find_new_shelf_prod(s_fix, init_alloc)
                update_alloc[s_change - 1] = p_new
                update_alloc[s_new - 1] = p_change


                new_result = q1_function(update_alloc)

                #update if its improved
                if new_result[0] < init_distance:
                    neighbourhoods.append(update_alloc)
                    neigh_dists.append(new_result[0])
                    #do we not also need to update init_distance??


    end_time = time.time()

    return neighbourhoods, np.array(neigh_dists), end_time - start_time

In [190]:
def LSH(allocation, max_k, time_limit):
    vns_it = 1
    init_allocations = allocation.copy()
    best_allocation = init_allocations
    best_distance = q1_function(best_allocation)[0]
    print(f"===== VNS iteration {vns_it} =====")
    neighs_to_explore, neigh_distances, neigh_time = swap_prods_worst_route_neighbourhoods(best_allocation, max_k)
    print(f"Time taken to find neighbourhoods: {neigh_time:.2f} seconds.")
    start_time = time.time()
    while time.time() - start_time < time_limit and len(neighs_to_explore) > 0:
        best_ind = np.argmin(neigh_distances)
        best_dist = neigh_distances[best_ind]
        vns_it += 1
        print(f"===== VNS iteration {vns_it} =====")
        best_allocation = neighs_to_explore[best_ind]
        neighs_to_explore, neigh_distances, neigh_time = swap_prods_worst_route_neighbourhoods(best_allocation, max_k)
        print(f"Time taken to find neighbourhoods: {neigh_time:.2f} seconds.")

    return best_allocation, time.time() - start_time

In [195]:
HeuristicAllocation = allocation_from_txt("mostCommonOrderAllocation.txt")
best_allocation, time_taken = LSH(HeuristicAllocation, max_k=10, time_limit=600)
print(time_taken)

===== VNS iteration 1 =====
Initial distance : 205038
Time taken to find neighbourhoods: 10.90 seconds.
===== VNS iteration 2 =====
Initial distance : 204504
Time taken to find neighbourhoods:  10.883132934570312
===== VNS iteration 3 =====
Initial distance : 204342
Time taken to find neighbourhoods:  11.100997924804688
===== VNS iteration 4 =====
Initial distance : 204096
Time taken to find neighbourhoods:  12.52268099784851
===== VNS iteration 5 =====
Initial distance : 203892
Time taken to find neighbourhoods:  13.63838815689087
===== VNS iteration 6 =====
Initial distance : 203754
Time taken to find neighbourhoods:  14.539716243743896
===== VNS iteration 7 =====
Initial distance : 203526
Time taken to find neighbourhoods:  12.70408582687378
===== VNS iteration 8 =====
Initial distance : 203280
Time taken to find neighbourhoods:  13.08171534538269
===== VNS iteration 9 =====
Initial distance : 203172
Time taken to find neighbourhoods:  13.206554889678955
===== VNS iteration 10 =====

In [197]:
best_allocation
HeuristicAllocation

array([34, 35, 18,  2, 50, 11, 48, 60, 19, 44, 47, 64, 74, 68, 69, 21, 88,
       14, 36, 53, 26,  3,  7,  0, 31, 71, 30, 58, 49, 28, 33, 43,  1, 59,
       66, 46, 65, 90, 16, 45, 27, 80, 75, 61, 40, 86, 42,  0, 70, 51, 67,
        4, 10, 52,  8, 63, 13, 72, 20, 32, 89, 82, 87, 24, 54,  9, 25, 62,
       73, 17,  0,  0, 15, 76,  6, 83, 38, 78, 55, 39, 57, 37, 84, 77, 79,
       29, 22,  5, 23, 56, 12, 41, 85, 81,  0,  0])

In [198]:
best_allocation, time_taken = LSH(Allocations.copy(), max_k=10, time_limit=1800)
print(time_taken)

===== VNS iteration 1 =====
Initial distance : 290706
Time taken to find neighbourhoods: 11.77 seconds.
===== VNS iteration 2 =====
Initial distance : 286110
Time taken to find neighbourhoods:  10.469883918762207
===== VNS iteration 3 =====
Initial distance : 281142
Time taken to find neighbourhoods:  9.99492597579956
===== VNS iteration 4 =====
Initial distance : 276624
Time taken to find neighbourhoods:  9.816861391067505
===== VNS iteration 5 =====
Initial distance : 273486
Time taken to find neighbourhoods:  9.874779224395752
===== VNS iteration 6 =====
Initial distance : 270738
Time taken to find neighbourhoods:  8.96588397026062
===== VNS iteration 7 =====
Initial distance : 268668
Time taken to find neighbourhoods:  8.699510097503662
===== VNS iteration 8 =====
Initial distance : 266598
Time taken to find neighbourhoods:  9.583911895751953
===== VNS iteration 9 =====
Initial distance : 264654
Time taken to find neighbourhoods:  9.363691091537476
===== VNS iteration 10 =====
Init